# HEASARC Weekly Photon Dataset HATS Import

Path to Data: https://heasarc.gsfc.nasa.gov/FTP/fermi/data/lat/weekly/photon/

Notebook Creation Date: May 30, 2025

### Installation shorthands

In [ ]:
#pip install --upgrade hats

In [ ]:
#cd ../../hats-import

In [ ]:
#pip install -e .

### Imports and path setup

In [ ]:
from dask.distributed import Client
from hats_import.pipeline import pipeline_with_client
from hats_import.catalog.arguments import ImportArguments
from astropy.io import fits
from astropy.table import Table
import glob


# Set data paths
epyc_path = "/astro/store/epyc3/data3/hats/tmp/heasarc_photon/weekly/photon"  # path to input data
output_path = "/astro/store/epyc3/data3/hats/catalogs/heasarc_photon"  # path of output catalog
#single_fits = "lat_photon_weekly_w009_p305_v001.fits"

# Gather file lists for import
subset = glob.glob(epyc_path+"/lat_photon_weekly_w01*.fits")  # small subset for testing
full = glob.glob(epyc_path+"/lat_photon_weekly_*.fits")  # full weekly

## hats-import setup and execution

In [ ]:
# Import arguments, using custom HeasarcReader

args = ImportArguments(
    sort_columns="EVENT_ID",
    ra_column="RA",
    dec_column="DEC",
    input_file_list=full,
    file_reader="fits",
    output_artifact_name="fermi_photon_catalog_with_index",
    output_path=f"{output_path}/heasarc_photon",
    dask_tmp="/astro/users/brantd/epyc/users/brantd/tmp/",  # setting a tmp directory in a known folder for dask is recommended when running on a distributed system
)


In [ ]:
# Run the import pipeline
with Client(n_workers=25, threads_per_worker=1, memory_limit="6GB") as client:
    
    pipeline_with_client(args, client)

## Viewing the resulting hats catalog

In [ ]:
import lsdb

fermi_hats = lsdb.read_hats(f"{output_path}/heasarc_photon/fermi_photon_catalog")
fermi_hats.plot_pixels()
fermi_hats

In [ ]:
fermi_hats.head(10)